In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import sklearn.metrics as skm
import sklearn.covariance as skc
import sklearn.svm as sks
import ensemble
from plotter import Plotter
import seaborn as sns

In [ ]:
n = 1000
df = pd.DataFrame(
    {
        'x': np.random.normal(size=n),
        'label': False
    }
)

anom = [5, 10, 20, 50]

anom_idx = np.random.choice(range(len(df)), size = len(anom), replace = False)

for i in range(len(anom)):
    df.iloc[anom_idx[i], df.columns.get_loc('x')] = anom[i]
    df.iloc[anom_idx[i], df.columns.get_loc('label')] = True
    
lof = skn.LocalOutlierFactor()
res = lof.fit(df[['x']])
df['score'] = np.abs(res.negative_outlier_factor_)

In [ ]:
df['score_std_norm'] = ensemble.Normalize().std_norm(df['score'])
df['score_minmax_norm'] = ensemble.Normalize().minmax_norm(df['score'])

In [ ]:
df.sort_values(by='score', ascending=False).head(10)

In [ ]:
plt.figure(figsize=(7,3))
sns.boxenplot(data = df, x = 'x')
plt.title('Test data - box plot')
plt.show()

In [ ]:
df['bin_score'] = np.random.choice([0,1], size=len(df), p = [0.9, 0.1])

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharey=True)
ax1.hist(np.log(df['score']), bins=20)
ax1.set_xlabel('log(score)')
ax1.set_ylabel('count')
ax1.set_title('Continuous scores')

ax2.bar(
    df['bin_score'].astype(str).value_counts().index,
    df['bin_score'].value_counts(),
    width=0.1
)
ax2.set_xlabel('value')
#ax2.set_ylabel('count')
ax2.set_title('Binary scores')

plt.show()

In [ ]:
df['score'].describe()

In [ ]:
df = ensemble.Ensemble().avg_ensemble(
    df, ['score_std_norm', 'bin_score'], ensemble_score='agg_with_std_norm'
)

df = ensemble.Ensemble().avg_ensemble(
    df, ['score_minmax_norm', 'bin_score'], ensemble_score='agg_with_minmax_norm'
)

In [ ]:
df.sort_values(by='score', ascending=False).head(10)

In [ ]:
Plotter.clustered_hmap(
    df,
    ['score', 'bin_score', 'agg_with_std_norm', 'agg_with_minmax_norm']
)